<a href="https://colab.research.google.com/github/mjgutierre/DataScience/blob/main/lab5_NLP_TextPrep_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Universidad EAFIT Maria jose Gutierrez Estrada 2024-2

Se cargaron las librerias necesarias

- nltk para 'procesamiento natural del lenguaje'
- pandas para procesamiento de dataframes, muy usado en preparación de datos
- re - expresiones regulares
- numpy, codecs, etc - otras



In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=29e8bc3b340fe97c5534d3f19721a9e7ab4768ca7672ac1c544da5a22b72cf70
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
# directorios (path) de entrada y salida:#
path_in="../datasets/"
path_out="../out/"
filenametxt_in='in.txt'
filenametxt_out='out.txt'
filenamecsv_in='in.csv'
filenamecsv_out='out.csv'

In [3]:
#create spark session, no requerido en AWS EMR/Spark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
df=spark.createDataFrame([(1,'I really liked this movie'),
                         (2,'I would recommend this movie to my friends'),
                         (3,'movie was alright but acting was horrible'),
                         (4,'I am never watching that movie ever again')],
                        ['user_id','review'])

In [5]:
df.show(5,False)

+-------+------------------------------------------+
|user_id|review                                    |
+-------+------------------------------------------+
|1      |I really liked this movie                 |
|2      |I would recommend this movie to my friends|
|3      |movie was alright but acting was horrible |
|4      |I am never watching that movie ever again |
+-------+------------------------------------------+



## Tokenization


In [6]:
from pyspark.ml.feature import Tokenizer

In [8]:
tokenization=Tokenizer(inputCol='review',outputCol='tokens')

In [9]:
tokenized_df=tokenization.transform(df)

In [10]:
tokenized_df.show(4,False)

+-------+------------------------------------------+---------------------------------------------------+
|user_id|review                                    |tokens                                             |
+-------+------------------------------------------+---------------------------------------------------+
|1      |I really liked this movie                 |[i, really, liked, this, movie]                    |
|2      |I would recommend this movie to my friends|[i, would, recommend, this, movie, to, my, friends]|
|3      |movie was alright but acting was horrible |[movie, was, alright, but, acting, was, horrible]  |
|4      |I am never watching that movie ever again |[i, am, never, watching, that, movie, ever, again] |
+-------+------------------------------------------+---------------------------------------------------+



## Stopwords removal


In [11]:
from pyspark.ml.feature import StopWordsRemover

In [12]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [13]:
refined_df=stopword_removal.transform(tokenized_df)

In [14]:
refined_df.select(['user_id','tokens','refined_tokens']).show(10,False)

+-------+---------------------------------------------------+----------------------------------+
|user_id|tokens                                             |refined_tokens                    |
+-------+---------------------------------------------------+----------------------------------+
|1      |[i, really, liked, this, movie]                    |[really, liked, movie]            |
|2      |[i, would, recommend, this, movie, to, my, friends]|[recommend, movie, friends]       |
|3      |[movie, was, alright, but, acting, was, horrible]  |[movie, alright, acting, horrible]|
|4      |[i, am, never, watching, that, movie, ever, again] |[never, watching, movie, ever]    |
+-------+---------------------------------------------------+----------------------------------+



## Movies reviews


In [21]:
text_df=spark.read.csv('sample_data/movie_reviews.csv',inferSchema=True,header=True,sep=',')
# datos desde S3

In [22]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [23]:
text_df.count()

7087

## Data Cleaning


In [24]:
tokenization=Tokenizer(inputCol='Review',outputCol='tokens')

In [25]:
tokenized_df=tokenization.transform(text_df)

In [26]:
tokenized_df.show()

+--------------------+---------+--------------------+
|              Review|Sentiment|              tokens|
+--------------------+---------+--------------------+
|The Da Vinci Code...|        1|[the, da, vinci, ...|
|this was the firs...|        1|[this, was, the, ...|
|i liked the Da Vi...|        1|[i, liked, the, d...|
|i liked the Da Vi...|        1|[i, liked, the, d...|
|I liked the Da Vi...|        1|[i, liked, the, d...|
|that's not even a...|        1|[that's, not, eve...|
|I loved the Da Vi...|        1|[i, loved, the, d...|
|i thought da vinc...|        1|[i, thought, da, ...|
|The Da Vinci Code...|        1|[the, da, vinci, ...|
|I thought the Da ...|        1|[i, thought, the,...|
|The Da Vinci Code...|        1|[the, da, vinci, ...|
|The Da Vinci Code...|        1|[the, da, vinci, ...|
|then I turn on th...|        1|[then, i, turn, o...|
|The Da Vinci Code...|        1|[the, da, vinci, ...|
|i love da vinci c...|        1|[i, love, da, vin...|
|i loved da vinci ...|      

In [27]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [28]:
refined_text_df=stopword_removal.transform(tokenized_df)

In [29]:
refined_text_df.show()

+--------------------+---------+--------------------+--------------------+
|              Review|Sentiment|              tokens|      refined_tokens|
+--------------------+---------+--------------------+--------------------+
|The Da Vinci Code...|        1|[the, da, vinci, ...|[da, vinci, code,...|
|this was the firs...|        1|[this, was, the, ...|[first, clive, cu...|
|i liked the Da Vi...|        1|[i, liked, the, d...|[liked, da, vinci...|
|i liked the Da Vi...|        1|[i, liked, the, d...|[liked, da, vinci...|
|I liked the Da Vi...|        1|[i, liked, the, d...|[liked, da, vinci...|
|that's not even a...|        1|[that's, not, eve...|[even, exaggerati...|
|I loved the Da Vi...|        1|[i, loved, the, d...|[loved, da, vinci...|
|i thought da vinc...|        1|[i, thought, da, ...|[thought, da, vin...|
|The Da Vinci Code...|        1|[the, da, vinci, ...|[da, vinci, code,...|
|I thought the Da ...|        1|[i, thought, the,...|[thought, da, vin...|
|The Da Vinci Code...|   

In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [31]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = refined_text_df.withColumn("token_count", len_udf(col('refined_tokens')))

In [32]:
refined_text_df.orderBy(rand()).show(10)

+--------------------+---------+--------------------+--------------------+-----------+
|              Review|Sentiment|              tokens|      refined_tokens|token_count|
+--------------------+---------+--------------------+--------------------+-----------+
|I hate Harry Potter.|        0|[i, hate, harry, ...|[hate, harry, pot...|          3|
|Its freezing cold...|        0|[its, freezing, c...|[freezing, cold, ...|          7|
|"Anyway, thats wh...|        1|["anyway,, thats,...|["anyway,, thats,...|          6|
|Brokeback Mountai...|        0|[brokeback, mount...|[brokeback, mount...|          4|
|Brokeback Mountai...|        1|[brokeback, mount...|[brokeback, mount...|          4|
|I love The Da Vin...|        1|[i, love, the, da...|[love, da, vinci,...|          4|
|I love Brokeback ...|        1|[i, love, brokeba...|[love, brokeback,...|          3|
|Harry Potter drag...|        0|[harry, potter, d...|[harry, potter, d...|          9|
|I want to be here...|        1|[i, want, t

In [42]:
from pyspark.sql.functions import col

# Convert the 'refined_tokens' column to a string representation
refined_text_df = refined_text_df.withColumn("refined_tokens", col("refined_tokens").cast("string"))

# Add the mode("overwrite") to the write command
refined_text_df.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save('.../Out/out.csv')